# Species classification by whistles, Oswald data
# Expt 1: DEPLOYMENT split;  cross-validation

# May  12, 2021

In [1]:
import numpy as np
from itertools import permutations
import random
import os
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import pandas as pd
from os import makedirs
from datetime import datetime
from collections import Counter

from math import floor

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

# from MulticoreTSNE import MulticoreTSNE as TSNE
# from sklearn.manifold import TSNE
import seaborn as sns

from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.layers import Conv2D, Lambda, Flatten, MaxPooling2D, LSTM, ConvLSTM2D, GlobalAveragePooling2D, GlobalMaxPooling2D  # Reshape, Lambda, Concatenate
from tensorflow.keras.layers import Bidirectional, LSTM, GRU
from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras import backend as K

from tensorflow.keras.optimizers.schedules import ExponentialDecay, PiecewiseConstantDecay
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy  # CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
# import tensorflow_addons.layers.spatial_pyramid_pooling as spp
# import tensorflow_datasets as tfds
from tensorflow.math import l2_normalize

import tensorflow as tf

from lib_validation import DataGenerator, find_best_model
from lib_model import model_cnn14_spp, model_cnn14_attention_multi

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# Hyper parameters

# 77.24 val for STAR2000
learning_rate = 1.e-4
conv_dim = 64
pool_size = 2
pool_stride = 2
l2_regu = 0.0001
drop_rate = 0.2
hidden_units = 512
fcn_dim = 512

run_num = 1

# # A little bit l2
# learning_rate = 1.e-4
# conv_dim = 64
# pool_size = 2
# pool_stride = 2
# l2_regu = 0.0001  ==>> good, but need fine-tuning
# l2_regu = 0.001   ==>> too much!
# drop_rate = 0.2
# hidden_units = 512
# fcn_dim = 512

num_epoch = 200
# num_epoch = 1  # debug
# batch_size = 128
batch_size = 32  # for cnn14+attention
# batch_size = 16  # for cnn14+spp
copies_of_aug =  10  # cannot be changed

num_patience = 20

In [17]:
# root_dir = '/home/ys587/__Data/__whistle'  # where we have __whislte_30_species folder
root_dir = '/home/ubuntu'  # where we have __whislte_30_species folder

work_path = os.path.join(root_dir, '__whistle_30_species')
fit_result_path =  os.path.join(work_path, '__fit_result_species')
# feature_path = os.path.join(work_path, '__feature_species')
# feature_path = os.path.join(work_path, '__dataset/20210210')

In [18]:
species_dict = {'BD': 0, 'CD': 1, 'STR': 2, 'SPT': 3, 'SPIN': 4, 'PLT': 5, 'RT': 6,  'FKW': 7}
num_species = len(species_dict)
species_list = list(species_dict.keys())
species_id = list(species_dict.values())

## Features

In [24]:
feature_path = os.path.join(root_dir, '__whistle_30_species/__dataset/__feature')

In [28]:
# df_species = pd.read_csv(os.path.join(feature_path, 'all.csv'))
df_species = pd.read_csv(os.path.join(feature_path, 'all_species.csv'))
df_noise = pd.read_csv(os.path.join(feature_path, 'all_noise.csv'))

## Split over deployments

In [29]:
today = datetime.now()
# create a folder based on date & time
# fit_result_path1 = os.path.join(fit_result_path, today.strftime('%Y%m%d_%H%M%S'))
fit_result_path1 = os.path.join(fit_result_path, today.strftime('%Y%m%d_%H%M%S')+'_deployment_run'+str(run_num))

In [30]:
# # debug
# df_species = df_species.sample(1000)
# df_noise = df_noise.sample(1000)

In [31]:
label_pred_all = []
label_test_all = []

## Model training & validation

In [ ]:
deploy_list = ['STAR2000', 'STAR2003', 'STAR2006', 'HICEAS2002', 'PICEAS2005']
random_list0 = [0, 10, 20, 30, 40]
random_list = [rr+run_num for rr in random_list0]

for ee0 in range(5):
    ee  = deploy_list[ee0]
    print(ee)

    # (a) testing, original data
    df_species_test = df_species[(df_species['deployment'] == ee)]
    print(df_species_test.shape)
    
    # loading
    fea_temp_orig = np.load(os.path.join(feature_path, 'all_orig.npz'))
    feas_orig = fea_temp_orig['feas_orig']
    labels_orig = fea_temp_orig['labels_orig']
    print('The shape of feas_orig: ', end='')
    print(feas_orig.shape)
    
    # original features & labels
    fea_ind_orig = np.array(df_species_test.index)
    fea_test = feas_orig[fea_ind_orig, :, :]
    label_test = labels_orig[fea_ind_orig]
    label_test = np.array([species_dict[ll] for ll in label_test])
    print(fea_test.shape)
    
    del feas_orig, labels_orig
    
    # (b) training
    df_species_train = df_species[(df_species['deployment'] != ee)]
    print(df_species_train.shape)
    
    # loading
    fea_temp_aug = np.load(os.path.join(feature_path, 'all_aug.npz'))
    feas_aug = fea_temp_aug['feas_aug']
    labels_aug = fea_temp_aug['labels_aug']
    print('The shape of feas_aug: ', end='')
    print(feas_aug.shape)
        
    # original features & labels
    fea_ind_orig = np.array(df_species_train.index)

    # augmented features & labels
    fea_ind_aug = []
    for ff in list(fea_ind_orig):
        for ii in range(copies_of_aug):
            fea_ind_aug.append(ff*copies_of_aug+ii)
        
    fea_train = feas_aug[fea_ind_aug, :, :]
    label_train = labels_aug[fea_ind_aug]
    label_train = np.array([species_dict[ll] for ll in label_train])
    print(fea_train.shape)
    
    del feas_aug, labels_aug
    
    # summary
    print('feature train shape: '+str(fea_train.shape))
    print('feature test shape: '+str(fea_test.shape))
    print('label train shape: '+str(label_train.shape))
    print('label test shape: '+str(label_test.shape))

    dim_time = fea_train.shape[1]
    dim_freq = fea_train.shape[2]
    print('dim_time: '+str(dim_time))
    print('dim_freq: '+str(dim_freq))
    
    # shuffle features & labels
    fea_train, label_train = shuffle(fea_train, label_train, random_state=random_list[ee0])
    fea_test, label_test = shuffle(fea_test, label_test, random_state=random_list[ee0])
    
    fea_train = np.expand_dims(fea_train, axis=3)
    fea_test = np.expand_dims(fea_test, axis=3)
    
    fea_train, fea_validate, label_train, label_validate = train_test_split(fea_train, label_train, test_size=0.10, random_state=random_list[ee0])

    train_generator = DataGenerator(fea_train, label_train, batch_size=batch_size, num_classes=num_species)
    del fea_train
    validate_generator = DataGenerator(fea_validate, label_validate, batch_size=batch_size, num_classes=num_species)
    del fea_validate
    
    # deployment folder
    fit_result_path2 = os.path.join(fit_result_path1, ee)
    if not os.path.exists(fit_result_path2):
        makedirs(fit_result_path2)
        
    # class weight
    weights = compute_class_weight(class_weight='balanced', classes=np.unique(label_train), y=label_train)

    class_weights = dict()
    for ii in range(num_species):
        class_weights[ii] = weights[ii]
        
    ### Training the model
    # model = model_cnn14_attention_multi(dim_time, dim_freq, num_species, model_type='feature_level_attention', conv_dim=conv_dim, pool_size=pool_size, pool_stride=pool_stride, hidden_units=hidden_units, l2_regu=l2_regu, drop_rate=drop_rate)
    model = model_cnn14_spp(dim_time, dim_freq, num_species, conv_dim=conv_dim, pool_size=pool_size, pool_stride=pool_stride, hidden_units=hidden_units, l2_regu=l2_regu, drop_rate=drop_rate)
    # loss = categorical_crossentropy
    loss = binary_crossentropy
    # model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_fn), loss=loss, metrics=['accuracy'])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=loss, metrics=['accuracy'])
    # model.summary()

    # With classes
    # history = model.fit(fea_train, to_categorical(label_train), class_weight=class_weights, validation_split=0.3, batch_size=batch_size, epochs=num_epoch, callbacks=[EarlyStopping(patience=num_patience), ModelCheckpoint(filepath=os.path.join(fit_result_path, '{epoch:02d}-{val_loss:.4f}.hdf5'), verbose=1, monitor="val_loss", save_best_only=True)])
    # history = model.fit(train_generator, validation_data=validate_generator, class_weight=class_weights, epochs=num_epoch, callbacks=[EarlyStopping(patience=num_patience, monitor='val_loss', mode='min', verbose=1), TensorBoard(log_dir=fit_result_path1), ModelCheckpoint(filepath=os.path.join(fit_result_path1, '{epoch:02d}-{val_loss:.4f}.hdf5'), verbose=1, monitor="val_loss", save_best_only=True)])
    # history = model.fit(train_generator, validation_data=validate_generator, class_weight=class_weights, epochs=num_epoch, callbacks=[EarlyStopping(patience=num_patience, monitor='val_loss', mode='min', verbose=1), TensorBoard(log_dir=fit_result_path2), ModelCheckpoint(filepath=os.path.join(fit_result_path2, 'epoch_{epoch:02d}_valloss_{val_loss:.4f}_valacc_{val_accuracy:.4f}.hdf5' ), verbose=1, monitor="val_loss", save_best_only=True)])
    # history = model.fit(train_generator, validation_data=validate_generator, class_weight=class_weights, epochs=num_epoch, callbacks=[EarlyStopping(patience=num_patience, monitor='val_loss', mode='min', verbose=1), TensorBoard(log_dir=fit_result_path2), ModelCheckpoint(filepath=os.path.join(fit_result_path2, 'epoch_{epoch:02d}_valloss_{val_loss:.4f}_valacc_{val_accuracy:.4f}.hdf5' ), verbose=1, monitor="val_accuracy", save_best_only=True)])
    history = model.fit(train_generator, validation_data=validate_generator, class_weight=class_weights, epochs=num_epoch, callbacks=[EarlyStopping(patience=num_patience, monitor='val_accuracy', mode='max', verbose=1), TensorBoard(log_dir=fit_result_path2), ModelCheckpoint(filepath=os.path.join(fit_result_path2, 'epoch_{epoch:02d}_valloss_{val_loss:.4f}_valacc_{val_accuracy:.4f}.hdf5' ), verbose=1, monitor="val_accuracy", save_best_only=True)])

    # Testing
    the_best_model, _ = find_best_model(fit_result_path2, purge=False)
    model = load_model(the_best_model)
    label_pred = model.predict(fea_test)

    # save the testing results
    np.savez(os.path.join(fit_result_path1, ee+'_test_results.npz'), label_test=label_test, label_pred=label_pred)
    
    label_pred_all.append(label_pred)
    label_test_all.append(label_test)
    
    del train_generator, validate_generator
    

STAR2000
(6706, 5)
The shape of feas_orig: (20074, 101, 128)
(6706, 101, 128)
(13368, 5)
The shape of feas_aug: (200740, 101, 128)
(133680, 101, 128)
feature train shape: (133680, 101, 128)
feature test shape: (6706, 101, 128)
label train shape: (133680,)
label test shape: (6706,)
dim_time: 101
dim_freq: 128
Epoch 1/200
3758/3759 [============================>.] - ETA: 0s - loss: 0.5772 - accuracy: 0.2743
Epoch 00001: val_accuracy improved from -inf to 0.40445, saving model to /home/ubuntu/__whislte_30_species/__fit_result_species/20210519_160101/STAR2000/epoch_01_valloss_0.4090_valacc_0.4045.hdf5
3759/3759 [==============================] - 191s 51ms/step - loss: 0.5771 - accuracy: 0.2743 - val_loss: 0.4090 - val_accuracy: 0.4045
Epoch 2/200
3758/3759 [============================>.] - ETA: 0s - loss: 0.3509 - accuracy: 0.4729
Epoch 00002: val_accuracy did not improve from 0.40445
3759/3759 [==============================] - 190s 51ms/step - loss: 0.3508 - accuracy: 0.4729 - val_loss:

In [ ]:
model.summary()

In [ ]:
label_pred_all = np.concatenate(label_pred_all)
label_test_all = np.concatenate(label_test_all)

In [ ]:
label_test_all.shape

## Confusion matrix

In [ ]:
np.set_printoptions(linewidth=200, precision=2, suppress=True)

In [ ]:
print("Confusion matrix:")
# cm = confusion_matrix(label_train[:label_train_pred.shape[0]], np.argmax(label_train_pred, axis=1), labels=species_id)
cm = confusion_matrix(label_test_all, np.argmax(label_pred_all, axis=1), labels=species_id)

print(species_list)
print('')
print(cm)
print('')

cm2 = cm*1.0
for ii in range(cm.shape[0]):
    cm_row = cm[ii, :]*1.0

    cm_row_sum = cm_row.sum()
    if cm_row_sum != 0:
        cm2[ii, :] = cm_row / cm_row_sum
    else:
        cm2[ii, :] = np.zeros(cm.shape[1])

print(cm2)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=species_list)
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm2, display_labels=species_list)

In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
disp.plot(include_values=True,
                     cmap='viridis', ax=ax, xticks_rotation='horizontal',
                     values_format=None, colorbar=True)


In [ ]:
fig, ax = plt.subplots(figsize=[15, 15])
disp2.plot(include_values=True,
                     cmap='viridis', ax=ax, xticks_rotation='horizontal',
                     values_format='.2f', colorbar=True)